In [126]:
import PIL
from PIL import Image
import os
import glob
from glob import glob
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

In [127]:
DATA_DIR = "/opt/ml/input/data/train/"
IMG_DIR = f"{DATA_DIR}/images"
DF_PATH = f"{DATA_DIR}/train.csv"

df = pd.read_csv(DF_PATH)

In [128]:
imgs=[]
image_dirs=[]

for path in df.path:
    image_dirs.append(os.path.join(IMG_DIR, path))
        
for path in image_dirs:
    for image in glob(f'{path}/**'):
        if not '.-' in image:
            imgs.append(image)

In [129]:
len(imgs)

18900

In [94]:
#!pip install facenet-pytorch
#!pip install retina-face

In [95]:
import torch 
import torch.nn as nn 
import pandas as pd 
import numpy as np 
import os
import cv2
from facenet_pytorch import MTCNN

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

workers = 0 if os.name == 'nt' else 4
print(workers)

Running on device: cuda:0
4


In [96]:
image_info = cv2.imread(imgs[0])
image_info.shape

(512, 384, 3)

In [97]:
img=cv2.imread(imgs[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
boxes, probs, landmarks = mtcnn.detect(img, landmarks=True)

print("box:", boxes) 
print("probability", probs)
print("landmarks", landmarks)

box: [[107.051865 138.58195  236.98534  307.61084 ]]
probability [0.99536705]
landmarks [[[141.22536 205.09787]
  [198.46149 211.60715]
  [155.66747 241.8305 ]
  [134.01852 274.95557]
  [178.16582 281.2558 ]]]


In [98]:
from retinaface import RetinaFace
import glob

In [99]:
# make empty dataframe
df_image = pd.DataFrame({})
cnt = 0 # intialize iteration count

# padding value before cropping
X_PADDING = 20 
Y_PADDING = 30 # gave more padding due to include mask on the chin & hair style

In [121]:
for index, row in tqdm(df.iterrows()):
    train_image_paths = []
    id = row["id"]
    gender = row["gender"]
    race = row["race"]
    age = row["age"]
    profile = row["path"]
    
    profile_path = os.path.join(IMG_DIR, profile)
    # print(profile_path)
    
    jpg_file_list = glob.glob(f"{profile_path}/*.jpg")
    jpeg_file_list = glob.glob(f"{profile_path}/*.jpeg")
    png_file_list = glob.glob(f"{profile_path}/*.png")
    list_images = jpg_file_list + jpeg_file_list + png_file_list
    
    for img_path in list_images:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        boxes, probs, landmarks = mtcnn.detect(img, landmarks=True)

        if probs[0]:
          # print("solved with mtcnn")
          # save face bounding box information
            xmin = int(boxes[0, 0]) - X_PADDING
            ymin = int(boxes[0, 1]) - Y_PADDING
            xmax = int(boxes[0, 2]) + X_PADDING
            ymax = int(boxes[0, 3]) + Y_PADDING

          # save landmark information
            left_eye = landmarks[0, 0]
            right_eye = landmarks[0, 1]
            mouth_left = landmarks[0, 2]
            mouth_right = landmarks[0, 3]
            nose = landmarks[0, 4]
        else:
            result_detected = RetinaFace.detect_faces(img_path)
            # print(result_detected)

            # try retinaface to resolve,:
            if type(result_detected) == dict:        
                #print("resolving with retinaface: ", img_path)
                # save face bounding box information
                xmin = int(result_detected["face_1"]["facial_area"][0]) - X_PADDING
                ymin = int(result_detected["face_1"]["facial_area"][1]) - Y_PADDING
                xmax = int(result_detected["face_1"]["facial_area"][2]) + X_PADDING
                ymax = int(result_detected["face_1"]["facial_area"][3]) + Y_PADDING

                # save landmark information
                face_landmarks = result_detected["face_1"]["landmarks"]
                left_eye = face_landmarks["left_eye"]
                right_eye = face_landmarks["right_eye"]
                mouth_left = face_landmarks["mouth_left"]
                mouth_right = face_landmarks["mouth_right"]
                nose = face_landmarks["nose"]

            # if both of the detection fails, center crop
            elif type(result_detected) == tuple:
                #print("this one is causing trouble: ", img_path)
                xmin = 80
                ymin = 50
                xmax = 80 + 220
                ymax = 50 + 320


                # leave landmark information empty
                face_landmarks = left_eye = right_eye = np.nan
                mouth_left = mouth_right = nose = np.nan

        # add row to the df_images with the extracted information
        df_image = df_image.append(
          {
            "id":id,
            "gender":gender,        
            "race":race,
            "age":age,        
            "path": profile,
            "img_path": img_path,
            "xmin": xmin,
            "ymin": ymin,
            "xmax": xmax,
            "ymax": ymax,
            "left_eye": left_eye,
            "right_eye": right_eye,
            "mouth_left": mouth_left,
            "mouth_right": mouth_right,
            "nose": nose
          }, ignore_index=True)

        # print data information every 100 iterations
        #if cnt % 100 == 0:
            #print(df_image.shape)
            #print(df_image.info())
            #print(df_image.tail())

df_image.head()

2700it [53:10,  1.18s/it]


,id,gender,race,age,path,img_path,xmin,ymin,xmax,ymax,left_eye,right_eye,mouth_left,mouth_right,nose
0,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,87.0,108.0,256.0,337.0,"[141.22536, 205.09787]","[198.46149, 211.60715]","[155.66747, 241.8305]","[134.01852, 274.95557]","[178.16582, 281.2558]"
1,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,90.0,128.0,266.0,350.0,"[148.73958, 220.77875]","[210.67368, 221.58511]","[176.10896, 244.32928]","[149.41524, 278.47006]","[198.27945, 280.38568]"
2,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,96.0,131.0,264.0,357.0,"[149.76358, 233.61348]","[208.68039, 233.47812]","[175.86276, 265.27472]","[149.84668, 291.00482]","[202.71521, 292.39996]"
3,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,99.0,105.0,261.0,322.0,"[152.33904, 206.31004]","[212.14357, 204.54936]","[181.62064, 238.68253]","[156.31369, 259.8815]","[206.18338, 258.45847]"
4,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,85.0,112.0,249.0,326.0,"[140.34601, 200.5359]","[196.44995, 203.56404]","[163.32722, 235.96231]","[140.95813, 266.59735]","[183.01239, 268.91034]"


In [122]:
df_image.to_csv(f"{DATA_DIR}/train_images.csv", index=False)

In [123]:
import pandas as pd 
import numpy as np 
import os
import cv2

# read training dataset
train_info = f"{DATA_DIR}/train_images.csv"
df_train_info = pd.read_csv(train_info)
df_train_info.head()

,id,gender,race,age,path,img_path,xmin,ymin,xmax,ymax,left_eye,right_eye,mouth_left,mouth_right,nose
0,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,87.0,108.0,256.0,337.0,[141.22536 205.09787],[198.46149 211.60715],[155.66747 241.8305 ],[134.01852 274.95557],[178.16582 281.2558 ]
1,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,90.0,128.0,266.0,350.0,[148.73958 220.77875],[210.67368 221.58511],[176.10896 244.32928],[149.41524 278.47006],[198.27945 280.38568]
2,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,96.0,131.0,264.0,357.0,[149.76358 233.61348],[208.68039 233.47812],[175.86276 265.27472],[149.84668 291.00482],[202.71521 292.39996]
3,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,99.0,105.0,261.0,322.0,[152.33904 206.31004],[212.14357 204.54936],[181.62064 238.68253],[156.31369 259.8815 ],[206.18338 258.45847]
4,006959,male,Asian,19.0,006959_male_Asian_19,/opt/ml/input/data/train//images/000001_female...,85.0,112.0,249.0,326.0,[140.34601 200.5359 ],[196.44995 203.56404],[163.32722 235.96231],[140.95813 266.59735],[183.01239 268.91034]


In [124]:
from tqdm import tqdm
import matplotlib.pyplot as plt 
import cv2

# make training folder
CROPPED_TRAIN_IMGS_DATASET_PATH = f"{DATA_DIR}/cropped_images"

if not os.path.exists(CROPPED_TRAIN_IMGS_DATASET_PATH):
    os.mkdir(CROPPED_TRAIN_IMGS_DATASET_PATH)

# iterrate rows for the given training dataset
for index, row in tqdm(df_image.iterrows()):
    # read stored data 
    img_path = row["img_path"]
    xmin = int(row["xmin"])
    ymin = int(row["ymin"])
    xmax = int(row["xmax"])
    ymax = int(row["ymax"])
    profile = str(row["path"])

    # make image profile folder
    profile_name = profile
    if not os.path.exists(os.path.join(CROPPED_TRAIN_IMGS_DATASET_PATH, profile_name)):
        os.mkdir(os.path.join(CROPPED_TRAIN_IMGS_DATASET_PATH, profile_name))
  
    # place cropped image under profile folder
    image_name = img_path.split("/")[-1]
    image = cv2.imread(img_path)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # omit BGR2RGB process: not using mtcnn

    # prevent tile out of box error
    if xmin<0:xmin=0
    if ymin<0:ymin=0
    if xmax>384:xmax=384
    if ymax>512:ymax=512

    # crop and save cropped images
    cropped_img = image[ymin:ymax, xmin:xmax, :] # y axis, x axis, all 3 channels
    # print(profile_name, image_name)
    cv2.imwrite(os.path.join(CROPPED_TRAIN_IMGS_DATASET_PATH, profile_name, image_name), cropped_img)

38331it [02:49, 225.67it/s]


In [131]:
imgs=[]
image_dirs=[]

for path in df.path:
    image_dirs.append(os.path.join(CROPPED_TRAIN_IMGS_DATASET_PATH, path))
        
for path in image_dirs:
    for image in glob(f'{path}/**'):
        if not '.-' in image:
            imgs.append(image)
len(image_dirs)
len(imgs)

18914